In [2]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes_transform.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes_transform.csv", index_col='Date', parse_dates=True)

In [4]:
dataset.loc['2024-06-10 9:30:00']

Open                         1311.7
High                         1313.1
Low                          1311.7
Close                        1312.4
Volume                         4718
time_d                   2024-06-10
P                            1305.2
R1                           1312.2
R2                           1316.4
R3                           1323.4
R4                           1330.4
R5                           1337.4
R6                           1344.4
S1                           1301.0
S2                           1294.0
S3                           1289.8
S4                           1285.6
S5                           1281.4
S6                           1277.2
First_Open                   1313.0
First_Close                  1311.5
Second_Open                  1311.6
Second_Close                 1311.2
Highest                      1314.0
Lowest                       1305.2
current         2024-06-10 09:35:00
prev_Close                   1311.9
ema_f                   1311

In [5]:
len(dataset)

74012

In [6]:
dataset.tail(3)

,Open,High,Low,Close,Volume,time_d,P,R1,R2,R3,...,ema_l,ema_l_shift,ma_20,price_std,upper_bb,lower_bb,rsi,cross_pivot,ema_cross,ibs
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-19 14:25:00,1317.3,1318.0,1314.3,1314.5,11350,2024-06-19,1313.133333,1317.766667,1327.033333,1331.666667,...,1309.539612,1309.142781,1308.525,6.373950,1321.272900,1295.777100,59.953184,NaN,NaN,0.054054
2024-06-19 14:30:00,1315.1,1315.1,1315.1,1315.1,371,2024-06-19,1313.133333,1317.766667,1327.033333,1331.666667,...,1309.951493,1309.539612,1309.190,6.329039,1321.848078,1296.531922,60.873747,NaN,NaN,1.000000
2024-06-19 14:45:00,1317.0,1317.0,1317.0,1317.0,7116,2024-06-19,1313.133333,1317.766667,1327.033333,1331.666667,...,1310.473605,1309.951493,1309.960,6.293723,1322.547446,1297.372554,63.717975,NaN,NaN,1.000000


## Volitility by hour

In [14]:
data_y = dataset.copy()
vol_by_hour = data_y.resample("H").agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min'
    })
vol_by_hour.dropna(inplace=True)
vol_by_hour['body'] = abs(vol_by_hour['Close'] - vol_by_hour['Open'])
vol_by_hour['height'] = vol_by_hour['High'] - vol_by_hour['Low']
grouped_y = vol_by_hour[['body', 'height']].groupby([vol_by_hour.index.hour]).mean()
grouped_y = grouped_y[grouped_y.body != 0]
grouped_y
# grouped_y['step_range'].sort_values()

,body,height
Date,,
9,3.200413,6.190496
10,3.145041,6.230441
11,2.416047,4.649311
13,4.272176,8.250895
14,5.221143,9.123898


## Try Fibonacci

In [ ]:
data_z = dataset.copy()
# data_z = data_z[100*data_z.index.hour + data_z.index.minute != 900]
data_z['r_threshold'] = data_z['First_Close'] + 6.18
data_z['s_threshold'] = data_z['First_Close'] - 6.18
data_z['cross_threshold'] = ''
data_z['return'] = 0

for i, row in data_z.iterrows():
    if row['prev_Close'] < row['r_threshold'] < row['Close']:
        # Long signal
        data_z.at[i, 'cross_threshold'] = 'cross_up'
        current_time = row.name
        current_date = current_time.strftime('%Y-%m-%d ').format()
        data_to_end_day = data_z[(data_z.index > current_time) & (data_z.index < current_date + ' 14:30:00')]
        if len(data_to_end_day):
            max_price = data_to_end_day['High'].max()
            last_item = data_to_end_day.iloc[-1]
            close_price = last_item['Close']
            if row['Close'] > max_price - 4:
                # neu ko bi stop thi se close deal luc 14:30
                data_z.at[i, 'return'] = row['Close'] - close_price
            else:
                data_z.at[i, 'return'] = max_price - 4 - row['Close']
        # No force stoploss
    elif row['prev_Close'] > row['s_threshold'] > row['Close']:
        # Short signal
        data_z.at[i, 'cross_threshold'] = 'cross_down'
        current_time = row.name
        current_date = current_time.strftime('%Y-%m-%d ').format()
        data_to_end_day = data_z[(data_z.index > current_time) & (data_z.index < current_date + ' 14:30:00')]
        if len(data_to_end_day):
            min_price = data_to_end_day['Low'].min()
            last_item = data_to_end_day.iloc[-1]
            close_price = last_item['Close']
            if row['Close'] > min_price + 6:
                # neu ko bi stop thi se close deal luc 14:30
                data_z.at[i, 'return'] = row['Close'] - close_price
            else:
                data_z.at[i, 'return'] = row['Close'] - min_price + 6
        # No force stoploss

In [ ]:
cross_data = data_z[data_z['cross_threshold'] != '']
day_cross_data = cross_data.resample("D").agg({
        'cross_threshold': 'count'
    })
day_cross_data.dropna(inplace=True)
day_cross_data.describe()

In [ ]:
cross_data

In [ ]:
len(cross_data[cross_data['return'] > 0]) / len(cross_data)

In [ ]:
cross_data[cross_data['return'] < 0]

## Signals from 9:15AM to 2:25PM

In [ ]:
# Muc tieu:
# Phan tich thoi gian tu 2:00PM - 2:25PM 
def cal_highest_price(tick):
    tick = tick[100*tick.index.hour+tick.index.minute > 1400]
    tick = tick[100*tick.index.hour+tick.index.minute < 1430]
    return tick.max()

def cal_lowest_price(tick):
    tick = tick[100*tick.index.hour+tick.index.minute > 1400]
    tick = tick[100*tick.index.hour+tick.index.minute < 1430]
    return tick.min()

def get_entry_price(tick):
    tick = tick[100*tick.index.hour+tick.index.minute == 1400]
    return tick

def get_exit_price(tick):
    tick = tick[100*tick.index.hour+tick.index.minute == 1425]
    return tick


data3 = dataset.copy()
data3['entry'] = data2['Open']
data3['exit'] = data2['Close']
data3['max_high'] = data2['High']
data3['min_low'] = data2['Low']
day_data = data3.resample("D").agg({
        'entry': get_entry_price,
        'exit': get_exit_price,
        'max_high': cal_highest_price,
        'min_low': cal_lowest_price
    })
day_data.tail(10)

In [ ]:
stoploss_day = day_data[(day_data.max_high - day_data.entry > 3.5) & (day_data.entry - day_data.min_low > 3.5)]
len(stoploss_day) / (len(day_data))
# Ket luan: cho co 6.6% gia vua tang vua giam 3.5 diem tu 2::00PM den 2:25PM

In [ ]:
long_day = day_data[day_data.exit > day_data.entry]
long_day

In [ ]:
short_day = day_data[day_data.exit < day_data.entry]
short_day

## Pivot points


In [ ]:
data = dataset.copy()
_back_days = 7
last_expire_date = date.today() - timedelta(days=_back_days)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
sample_data = data[(data.index > last_expire_date) & (data.index < date.today().strftime("%Y-%m-%d 23:00:00"))]
cross_up_data = sample_data[(sample_data.cross_pivot == 'cross_up') & (sample_data.ema_f > sample_data.ema_l)]
cross_up_data

##Momentum(01) Analytics

In [ ]:
# data_4momentum = dataset.copy()
# _back_days = 7
# last_expire_date = date.today() - timedelta(days=_back_days)
# last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
# data_4momentum = data_4momentum[(data_4momentum.index > last_expire_date) & (data_4momentum.index < date.today().strftime("%Y-%m-%d 23:00:00"))]
# data_4momentum['First_Close'] = data_4momentum['First_Close'].astype(float)
# data_4momentum['momentum'] = data_4momentum['Close'] - data_4momentum['First_Close']
# t = data_4momentum['momentum'].to_numpy()
# plt.plot(t)
# plt.show()

## Momentum(02) Analytics

In [ ]:
# data_4momentum = dataset.copy()
# _back_days = 2
# last_expire_date = date.today() - timedelta(days=_back_days)
# last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
# data_4momentum = data_4momentum[(data_4momentum.index > last_expire_date) & (data_4momentum.index < date.today().strftime("%Y-%m-%d 00:00:00"))]
# data_4momentum['baseline'] = data_4momentum.groupby(pd.Grouper(freq='1D'))['Open'].transform('first')
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# data_4momentum['value'] = data_4momentum['Close'] * data_4momentum['Volume']
# steps =  [*range(0, len(data_4momentum), 1)]
# data_4momentum = data_4momentum.assign(steps=steps)
# data_4momentum['sum_value'] = data_4momentum['value'].cumsum()
# data_4momentum['momentum'] = data_4momentum.apply(
# lambda row: (0 if row.steps == 0 else round(row.sum_value / (row.steps * 1000))), axis=1)
# data_4momentum.tail(10)

## Relationship between First 5 min and whole day

In [ ]:
# def cal_first_open(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 900]
#   return tick

# def cal_first_close(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 900]
#   return tick

# data2 = prepared_data.copy()
# data2['First_Open'] = data2['Open']
# data2['First_Close'] = data2['Close']
# day_data = data2.resample("D").agg({
#         'Open': 'first',
#         'Close': 'last',
#         'High': 'max',
#         'Low': 'min',
#         'Volume': 'sum',
#         'First_Open': cal_first_open,
#         'First_Close': cal_first_close
#     })
# day_data['diff_up'] = day_data['High'] - day_data['First_Open']
# day_data['diff_down'] = day_data['First_Open'] - day_data['Low']
# day_data['first_green'] = day_data.apply(lambda row: 1 if row['First_Open'] < row['First_Close'] else 0, axis =1)
# day_data['up_bigger_down'] = day_data.apply(lambda row: 1 if row['diff_up'] > row['diff_down'] else 0, axis =1)
# day_data.dropna(inplace=True)
# same = day_data[day_data.first_green == day_data.up_bigger_down]
# diff = day_data[day_data.first_green != day_data.up_bigger_down]
# same_count = len(same)
# diff_count = len(diff)
# same_p = same_count/(diff_count+same_count)
# diff_p = diff_count/(diff_count+same_count)
# y = np.array([same_p, diff_p])
# labels = ["Same({})".format(same_p), "Different({})".format(diff_p)]

# plt.pie(y, labels = labels)
# plt.show()


##Relationship between 2 first candlesticks and whole day

In [ ]:
# def cal_first_open(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 900]
#   return tick

# def cal_second_open(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 905]
#   return tick

# def cal_first_close(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 900]
#   return tick
# def cal_second_close(tick):
#   tick = tick[100*tick.index.hour+tick.index.minute == 905]
#   return tick

# data2 = prepared_data.copy()
# _back_days = 30
# last_expire_date = date.today() - timedelta(days=_back_days)
# last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
# data2 = data2[data2.index > last_expire_date]
# data2['First_Open'] = data2['Open']
# data2['First_Close'] = data2['Close']
# data2['Second_Open'] = data2['Open']
# data2['Second_Close'] = data2['Close']
# day_data = data2.resample("D").agg({
#         'Open': 'first',
#         'Close': 'last',
#         'High': 'max',
#         'Low': 'min',
#         'Volume': 'sum',
#         'First_Open': cal_first_open,
#         'First_Close': cal_first_close,
#         'Second_Open': cal_second_open,
#         'Second_Close': cal_second_close
#     })
# # day_data['diff_up'] = day_data['High'] - day_data['First_Open']
# # day_data['diff_down'] = day_data['First_Open'] - day_data['Low']
# day_data['first_green'] = day_data.apply(lambda row: 1 if row['First_Open'] <= row['First_Close'] else 0, axis =1)
# day_data['second_green'] = day_data.apply(lambda row: 1 if row['Second_Open'] <= row['Second_Close'] else 0, axis =1)
# day_data[(day_data.first_green != day_data.second_green)]
# # day_data['up_bigger_down'] = day_data.apply(lambda row: 1 if row['diff_up'] > row['diff_down'] else 0, axis =1)
# # day_data.dropna(inplace=True)
# # same = day_data[(day_data.first_green == day_data.second_green) & (day_data.first_green == day_data.up_bigger_down)]
# # diff = day_data[(day_data.first_green == day_data.second_green) & (day_data.first_green) != day_data.up_bigger_down]
# # same_count = len(same)
# # diff_count = len(diff)
# # same_p = same_count/(diff_count+same_count)
# # diff_p = diff_count/(diff_count+same_count)
# # y = np.array([same_p, diff_p])
# # labels = ["Same({})".format(same_p), "Different({})".format(diff_p)]

# # plt.pie(y, labels = labels)
# # plt.show()

## Find all long upper shadow & lower shadow


In [ ]:
# candlestick_data = prepared_data.copy()
# candlestick_data['has_long_upper_shadow'] = candlestick_data.apply(lambda row: True if (row['High'] - max(row['Close'], row['Open']) > 2) else False, axis=1)
# candlestick_data['has_long_lower_shadow'] = candlestick_data.apply(lambda row: True if (min(row['Close'], row['Open']) - row['Low'] > 2) else False, axis=1)
# candlestick_data[(candlestick_data.has_long_upper_shadow == True) | (candlestick_data.has_long_lower_shadow == True)]

## Phan tich gia Open cao nhat va thap nhat trong khoang thoi gian 9h15->14h15

In [ ]:
# def cal_is_max_open(col, df):
#   return col.name.h
# data3 = prepared_data.copy()
# _back_days = 30
# data3['is_max_open'] = data3['Open'].apply(lambda x: cal_is_max_open(x, data3), axis=0)
# data3
